# PhoneNow - Call Centre Report

In this task, I'm a data analyst consultant for **PhoneNow** - a big telecom company. I've just received an email from Claire, a Call Centre Manager, she is looking for transparency and insight into the data we have at the Call Centre; for example, total number of calls answered and abandoned, speed of answer, length of calls, overall satisfaction etc. She just wants an accurate overview of long-term trends in customer and agent behaviour.

The goal is therefore to create a dashboard on Call Centre trends that she can use as a basis for discussion with management.

## Strategy
To takle this problem, I'm first going to perform exploratory analysis in this notebook, find patterns that can actually go into a dashboard, them implement them in Power BI.

Looking at the data provided by Claire, I think it makes sense to break down the metrics and KPIs into 3 categories:
- **Customer Experience**
- **Call Volume and Efficiency**
- **Agent Performance**


In [1]:
# DATA MANIPULATION
import os
import copy
import pandas as pd
import numpy as np

# DATA VIZ
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("fivethirtyeight")
# plt.rcParams["figure.figsize"] = [8, 5]
# plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings

warnings.filterwarnings("ignore")
sns.set(color_codes=True)

In [2]:
PHONENOW_COLOURS = [
    "#0072CE",
    "#B4B4B3",
    "#79B8F3",
    "#FDB927",
    "#F7941D",
    "#4CB748",
    "#2E3192",
]
DIVERGENT_COLOUR_GRADIENT = [
    "#e2f1fc",
    "#b9dcfa",
    "#8cc7f7",
    "#5eb1f3",
    "#39a0f1",
    "#0691ef",
]
sns.set_palette(PHONENOW_COLOURS)

# Loading and Cleaning Data

In [3]:
df = pd.read_csv("./call-centre-dataset.csv")
df.head()

,Call Id,Agent,Date,Time,Topic,Answered (Y/N),Resolved,Speed of answer in seconds,AvgTalkDuration,Satisfaction rating
0,ID0001,Diane,2021-01-01,9:12:58,Contract related,Y,Y,109.0,0:02:23,3.0
1,ID0002,Becky,2021-01-01,9:12:58,Technical Support,Y,N,70.0,0:04:02,3.0
2,ID0003,Stewart,2021-01-01,9:47:31,Contract related,Y,Y,10.0,0:02:11,3.0
3,ID0004,Greg,2021-01-01,9:47:31,Contract related,Y,Y,53.0,0:00:37,2.0
4,ID0005,Becky,2021-01-01,10:00:29,Payment related,Y,Y,95.0,0:01:00,3.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Call Id                     5000 non-null   object 
 1   Agent                       5000 non-null   object 
 2   Date                        5000 non-null   object 
 3   Time                        5000 non-null   object 
 4   Topic                       5000 non-null   object 
 5   Answered (Y/N)              5000 non-null   object 
 6   Resolved                    5000 non-null   object 
 7   Speed of answer in seconds  4054 non-null   float64
 8   AvgTalkDuration             4054 non-null   object 
 9   Satisfaction rating         4054 non-null   float64
dtypes: float64(2), object(8)
memory usage: 390.8+ KB


In [5]:
df = df.rename(
    columns={
        "Call Id": "call_id",
        "Agent": "agent",
        "Date": "date",
        "Time": "time",
        "Topic": "topic",
        "Answered (Y/N)": "answered",
        "Resolved": "resolved",
        "Speed of answer in seconds": "answerSpeed",
        "AvgTalkDuration": "avgTalkDuration",
        "Satisfaction rating": "rating",
    }
)

df["date"] = pd.to_datetime(df["date"])
df["time"] = pd.to_datetime(df["time"]).dt.time
df["answered"] = df["answered"].map({"Y": 1, "N": 0})
df["resolved"] = df["resolved"].map({"Y": 1, "N": 0})
df["avgTalkDuration"] = pd.to_datetime(df["avgTalkDuration"]).dt.time
df["rating"] = df["rating"].astype("Int64")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   call_id          5000 non-null   object        
 1   agent            5000 non-null   object        
 2   date             5000 non-null   datetime64[ns]
 3   time             5000 non-null   object        
 4   topic            5000 non-null   object        
 5   answered         5000 non-null   int64         
 6   resolved         5000 non-null   int64         
 7   answerSpeed      4054 non-null   float64       
 8   avgTalkDuration  4054 non-null   object        
 9   rating           4054 non-null   Int64         
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 395.6+ KB


In [7]:
df.head()

,call_id,agent,date,time,topic,answered,resolved,answerSpeed,avgTalkDuration,rating
0,ID0001,Diane,2021-01-01,09:12:58,Contract related,1,1,109.0,00:02:23,3
1,ID0002,Becky,2021-01-01,09:12:58,Technical Support,1,0,70.0,00:04:02,3
2,ID0003,Stewart,2021-01-01,09:47:31,Contract related,1,1,10.0,00:02:11,3
3,ID0004,Greg,2021-01-01,09:47:31,Contract related,1,1,53.0,00:00:37,2
4,ID0005,Becky,2021-01-01,10:00:29,Payment related,1,1,95.0,00:01:00,3


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   call_id          5000 non-null   object        
 1   agent            5000 non-null   object        
 2   date             5000 non-null   datetime64[ns]
 3   time             5000 non-null   object        
 4   topic            5000 non-null   object        
 5   answered         5000 non-null   int64         
 6   resolved         5000 non-null   int64         
 7   answerSpeed      4054 non-null   float64       
 8   avgTalkDuration  4054 non-null   object        
 9   rating           4054 non-null   Int64         
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 395.6+ KB


# Customer Experience
In this section we're going to focus on metrics that we'd think about when improving customer experience, which are:
- [x] What challenges prompt customer calls(topics)?
- [ ] What's the resolution rate for each of these challenges?
- [ ] How long do these calls take per topic?
- [ ] How are the rating scores per topic?

## Topics

In [9]:
df['topic'].unique()

array(['Contract related', 'Technical Support', 'Payment related',
       'Admin Support', 'Streaming'], dtype=object)

In [10]:
topic_counts = df["topic"].value_counts()
topic_counts.head()

topic
Streaming            1022
Technical Support    1019
Payment related      1007
Contract related      976
Admin Support         976
Name: count, dtype: int64

There's practically no difference on the number of calls per topic; in other words we don't see anything alarming as yet. Perhaps we can check if there was a time when one topic had numbers we're not used to.

In [11]:
fig = px.bar(
    x=topic_counts.index,
    y=topic_counts.values,
    labels={"x": "Topics", "y": "Number of Calls"},
    title="Number of Calls per Topic",
)
fig.show()

In [12]:
daily_calls_per_topic = (
    df.groupby(["date", "topic"])
    .size()
    .to_frame(name="count")
    .reset_index()
    .sort_values(by=["date", "topic"])
)

In [13]:
daily_calls_per_topic.head(10)

,date,topic,count
0,2021-01-01,Admin Support,10
1,2021-01-01,Contract related,11
2,2021-01-01,Payment related,11
3,2021-01-01,Streaming,10
4,2021-01-01,Technical Support,16
5,2021-01-02,Admin Support,8
6,2021-01-02,Contract related,14
7,2021-01-02,Payment related,7
8,2021-01-02,Streaming,11
9,2021-01-02,Technical Support,20


In [14]:
fig = go.Figure()
for topic in daily_calls_per_topic["topic"].unique():
    data_subset = daily_calls_per_topic[daily_calls_per_topic["topic"] == topic]
    fig.add_trace(
        go.Scatter(
            x=data_subset["date"],
            y=data_subset["count"],
            mode="lines",
            name=topic,
        )
    )

# Customize the graph layout
fig.update_layout(
    title="Number of Calls per Day by Topic",
    xaxis_title="Date",
    yaxis_title="Number of Calls",
    legend_title="Topic",
)

# Display the graph
fig.show()

We learn from these that:
- Admin Support & Contract related: No discernible pattern 
- Payment related: Calls under this topic seem to have cycles that repeat every 4, 5 or 6 days which I assume suggests a weekly pattern, therefore, let's aggregate calls by day of week.
- Technical Support: They seem to have a monthly cycle

In [21]:
df["date"] = pd.to_datetime(df["date"])
df["dayOfWeek"] = df["date"].dt.day_name()
payment_related_calls = df[df["topic"] == "Payment related"]

callsByDayOfWeek = (
    payment_related_calls.groupby("dayOfWeek")["call_id"]
    .count()
    .reset_index()
    .sort_values(by="call_id")
)

In [22]:
callsByDayOfWeek.head()

,dayOfWeek,call_id
0,Friday,132
4,Thursday,133
5,Tuesday,142
1,Monday,145
6,Wednesday,149


In [23]:
fig = px.bar(
    callsByDayOfWeek,
    x="dayOfWeek",
    y="call_id",
    title="Number of Calls Related to Payment per Day of the Week",
    labels={"dayOfWeek": "Day of the Week", "call_id": "Number of Calls"},
)
fig.show()

In [36]:
df["dayOfMonth"] = df["date"].dt.day
techSupportCalls = df[
    df["topic"] == "Technical Support"
]

techCallsByDayOfMonth = (
    techSupportCalls.groupby("dayOfMonth")["call_id"]
    .count()
    .reset_index()
    .sort_values(by="dayOfMonth")
)

In [37]:
techCallsByDayOfMonth.head()

,dayOfMonth,call_id
0,1,43
1,2,34
2,3,38
3,4,43
4,5,38


In [38]:
fig = px.line(
    techCallsByDayOfMonth,
    x="dayOfMonth",
    y="call_id",
    title="Number of Calls Related to Technical Support per Day of the Month",
    labels={"dayOfMonth": "Day of the Month", "call_id": "Number of Calls"},
)
fig.show()

## Topic Resolutions

In [40]:
resolved_counts = df.groupby(["topic", "resolved"]).size().reset_index(name="count")
resolved_counts.head()

,topic,resolved,count
0,Admin Support,0,253
1,Admin Support,1,723
2,Contract related,0,267
3,Contract related,1,709
4,Payment related,0,278


In [45]:
fig = px.bar(
    resolved_counts,
    x="topic",
    y="count",
    color="resolved",
    title="Resolved and Unresolved Calls per Topic",
    labels={"topic": "Topic", "count": "Number of Calls", "resolved": "Resolved"},
    color_continuous_scale=PHONENOW_COLOURS
)
fig.update_layout(barmode="stack")
fig.show()

So almost 20% of calls per topic aren't resolved

In [43]:
rating_counts = df.groupby(["topic", "rating"]).size().reset_index(name="count")
rating_counts.head()

,topic,rating,count
0,Admin Support,1,72
1,Admin Support,2,76
2,Admin Support,3,259
3,Admin Support,4,217
4,Admin Support,5,171


In [46]:
fig = px.bar(
    rating_counts,
    x="topic",
    y="count",
    color="rating",
    title="Distribution of Ratings per Topic",
    labels={"topic": "Topic", "count": "Number of Calls", "rating": "Rating"},
    color_continuous_scale=PHONENOW_COLOURS,
)

fig.update_layout(barmode="stack")

fig.show()

Since there's nothing interesting about drilling down into specific topics, our dashboard will only have overall aggregates like we do below:

In [47]:
resolved_percentage = df["resolved"].mean() * 100
unresolved_percentage = 100 - resolved_percentage

resolution_df = pd.DataFrame(
    {
        "Resolution Status": ["Resolved", "Unresolved"],
        "Percentage": [resolved_percentage, unresolved_percentage],
    }
)

resolution_df.head()

,Resolution Status,Percentage
0,Resolved,72.92
1,Unresolved,27.08


In [50]:
rating_percentages = (
    df["rating"]
    .value_counts(normalize=True)
    .sort_index() * 100)

rating_percentages_df = pd.DataFrame(
    {
        "Rating": rating_percentages.index,
        "Percentage": rating_percentages.values
    }
)

rating_percentages_df.head()

,Rating,Percentage
0,1,10.286137
1,2,9.76813
2,3,30.044401
3,4,29.107055
4,5,20.794277


In [61]:
resolution_fig = px.bar(
    resolution_df,
    x="Resolution Status",
    y="Percentage",
    color="Resolution Status",
    labels={"Resolution Status": "Resolution Status", "Percentage": "Percentage"},
    title="Resolution Rates",
    color_continuous_scale=PHONENOW_COLOURS,
    width=300,
)

fig_rating_percentages = px.bar(
    x=rating_percentages.index,
    y=rating_percentages.values,
    labels={"x": "Rating", "y": "Percentage"},
    title="Rating Percentages",
    width=700,
)

fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Resolution Rates", "Rating Percentages")
)

fig.add_trace(resolution_fig["data"][0], row=1, col=1)
fig.add_trace(fig_rating_percentages["data"][0], row=1, col=2)

fig.update_layout(width=1100, showlegend=False)

fig.show()

Okay, now we have a bug. Will deal with it later